In [4]:
import os
from dotenv import load_dotenv, find_dotenv

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[:3])
     

['/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/010-奥修禅塔罗牌（版本一）.txt', '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/010-奥修禅塔罗牌[附图]（版本二）.txt (010-奥修禅塔罗牌[附图]（版本二）.txt) (Z-Library).txt', '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/清水玲子塔罗 解説書／G.タトテ研究所 （牌意部分）.pdf (清水玲子塔罗 解説書／G.タトテ研究所 （牌意部分）.pdf) (Z-Library).pdf']


In [18]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import TextLoader
# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))
    elif file_type == 'txt':
        loaders.append(TextLoader(file_path, encoding='gb18030'))

In [19]:
loaders

In [20]:
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())

In [21]:
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/010-奥修禅塔罗牌[附图]（版本二）.txt (010-奥修禅塔罗牌[附图]（版本二）.txt) (Z-Library).txt'}
------
查看该文档的内容:


　　奥修禅塔罗牌

　　奥修禅塔罗牌介绍

　　引        言

　　在奥修对门徒及朋友们三十年的演讲期间，奥修会回答他们的问题，或者是评论世界上伟大的圣贤们的教导和经典。他的演讲持续地将新鲜的洞见带到每一件事上面，从晦涩的梵文经典到耳热能详的耶稣的话语全部包括。他在谈论哈希德派、苏菲宗派、鲍尔神秘家、瑜伽、和佛学时都带着同样的权威。最后,奥修集中在传达禅宗的独特智慧，因为，他说:禅宗是一个重要的心灵传统，它对人类内在生活的处理方式经得起时间的考验，它跟现代人仍然有着密不可分的关系。

　　不像其他的宗教或多或少都沦为英雄崇拜或教条主义，禅宗坚持说每一个人都有能力成道，或者可以有免于由头脑所创造出来的幻象的自由。禅宗坚持说这个能力只能够透过静心而达成。不是藉着遵循某种仪式，或是固守一套规则，或是模仿别人的例子，而是藉着对一个人自己的思想、行动、和感觉保持高度的警觉和不判断。当我们在练习这种警觉、这种面对人生的静心方式，它很快地就会使我们觉知到：我们每一个人都包含着一个不变的、不会受到打扰的：永恒的观照中心，那个中心具有一种能力，可以将人生看成一个伟大的冒险、一个游戏、一个神秘学校，以及，最后，一个喜乐的旅程，除了途中每一步都高高兴兴地去经历之外没有其他的目的。

　　在奥修的话语里面，重点并不是去崇拜诸佛，而是去变成一个佛；不是去遵循别人，而是去发展觉知，在那个当中将一种光和爱的品质带到我们所做的每一件事上面。

　　然而奥修一直都并非只是一个演说家或哲学家。在他的社区里，他创造出一种气氛，在那里，他所说的真理能够被直接经验到。去经验奥修的途径并不是速离市场而隐居到深山里面去过着苦行和休息的生活”相反地，它是完全投入在醒悟的任务里----透过各种静心、身心的治疗、很多欢笑和游戏、以及创造性

In [41]:
import getpass
import os
if not os.getenv("ZHIPUAI_API_KEY"):
    os.environ["ZHIPUAI_API_KEY"] = getpass.getpass("Enter your ZhipuAI API key: ")

In [ ]:
# 使用 OpenAI Embedding
# from langchain.embeddings.openai import OpenAIEmbeddings
# 使用百度千帆 Embedding
# from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint
# 使用我们自己封装的智谱 Embedding，需要将封装代码下载到本地使用
from zhipuai_embedding import ZhipuAIEmbeddings

# 定义 Embeddings
# embedding = OpenAIEmbeddings() 
embedding = ZhipuAIEmbeddings()
# embedding = QianfanEmbeddingsEndpoint()

# 定义持久化路径
persist_directory = '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming'

In [55]:
# ========== 3. 切分文档（核心：生成 split_docs） ==========
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,          # 每个片段500字符（中文适配）
    chunk_overlap=50,        # 片段间重叠50字符（保证上下文连贯）
    separators=["\n\n", "\n", "。", "！", "？", "；", "，"]  # 中文优先切分符
)
split_docs = text_splitter.split_documents(texts)  # 切分后的文档片段
print(f"文档切分完成，片段数：{len(split_docs)}")

文档切分完成，片段数：513


In [59]:
from langchain_community.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [60]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：513


In [61]:
question="如何算命"

In [62]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [63]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
算命大法[推荐]
第一种，刀切斧断型
    这类命师，算命水平较高，或自己认为自己很高，自我感觉良好，给人算命时，速度很快，没有术语，没有分析，或人家看不到他在分析，只有结果，不论是好是坏，硬梆梆的几句话，向求算者端出去就是。比如前面那位算天罡八字的王某，一次给某老太婆算命，八字还没排完，语言就出来了：“你八字克夫，嫁一个，克死一个，嫁二个，克死二个，嫁三个，还是克死。儿女三四个，最后孤身一人，
--------------
检索到的第1个内容: 
第二种，思想工作型
    这类命师，本身水平一般，故而算命时废话较多，从甲木的特性到乙木的特性，再到癸水的特性，如甲木代表参天大木，栋梁之材，乙木代表花草灌木，柔弱多情，正财代表工资，偏财代表生意的财、打麻将的财等等，罗里罗嗦一大堆，听他算命，要两个小时以上，而且什么知识都学到了。这类思想工作型命师，讲废话的时候是在思考分析，多数要含沙射影，旁敲侧击，察言观色，想办法套人家的东东出来，然后对症下
--------------
检索到的第2个内容: 
先天后天论
--------------


In [64]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [65]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
算命大法[推荐]
第一种，刀切斧断型
    这类命师，算命水平较高，或自己认为自己很高，自我感觉良好，给人算命时，速度很快，没有术语，没有分析，或人家看不到他在分析，只有结果，不论是好是坏，硬梆梆的几句话，向求算者端出去就是。比如前面那位算天罡八字的王某，一次给某老太婆算命，八字还没排完，语言就出来了：“你八字克夫，嫁一个，克死一个，嫁二个，克死二个，嫁三个，还是克死。儿女三四个，最后孤身一人，
--------------
MMR 检索到的第1个内容: 
先天后天论
--------------
MMR 检索到的第2个内容: 
新十二星座分析
--------------
